# 1. Requirements and first setup

In [2]:
!pip install mediapipe opencv-python pandas scikit-learn

In [1]:
import mediapipe as mp
import cv2 

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [5]:
cap = cv2.VideoCapture(0)

#Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # Make Detections
        results = holistic.process(image)
        
        # Recolor back
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS, 
                                  mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                  mp_drawing.DrawingSpec(color=(80,256,121), thickness=1,circle_radius=1)
                                 )
        # Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                  mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(80,256,121), thickness=2,circle_radius=2)
                                 )
        # Left hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                  mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(80,256,121), thickness=2,circle_radius=2)
                                 )
        # Pose landmarks
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                  mp_drawing.DrawingSpec(color=(245,117,10), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(245,256,121), thickness=2,circle_radius=2)
                                 )
        
        cv2.imshow('Raw webcam Feed', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    
cap.release()
cv2.destroyAllWindows()        

In [11]:
results

mediapipe.python.solution_base.SolutionOutputs

# 2. Capture landmarks and create CSV

In [3]:
import csv
import os
import numpy as np

In [6]:
num_coords = len(results.face_landmarks.landmark) + len(results.pose_landmarks.landmark)
num_coords

501

In [7]:
# creating the landmarks format
landmarks = ['class']
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [13]:
# create the csv file
with open('coords.csv', mode='w', newline='')as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting = csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [14]:
# Class name you're trainig
class_name = "Happy"

In [15]:
cap = cv2.VideoCapture(0)

#Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # Make Detections
        results = holistic.process(image)
        
        # Recolor back
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS, 
                                  mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                  mp_drawing.DrawingSpec(color=(80,256,121), thickness=1,circle_radius=1)
                                 )
        # Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                  mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(80,256,121), thickness=2,circle_radius=2)
                                 )
        # Left hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                  mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(80,256,121), thickness=2,circle_radius=2)
                                 )
        # Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                  mp_drawing.DrawingSpec(color=(245,117,10), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(245,256,121), thickness=2,circle_radius=2)
                                 )
        # Export coordinates
        try:
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            row = pose_row + face_row
            row.insert(0, class_name)
            
            with open('coords.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row)
            
        except:
            pass
        
        
        cv2.imshow('Raw webcam Feed', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    
cap.release()
cv2.destroyAllWindows() 

In [76]:
row

[0.377088725566864,
 0.4000910520553589,
 -0.8494803309440613,
 0.9998297691345215,
 0.41410237550735474,
 0.33231469988822937,
 -0.8451960682868958,
 0.9997959733009338,
 0.43560677766799927,
 0.33191949129104614,
 -0.8449777364730835,
 0.99967360496521,
 0.4579119086265564,
 0.33251604437828064,
 -0.8457099199295044,
 0.9997970461845398,
 0.36254385113716125,
 0.3362950086593628,
 -0.7862470746040344,
 0.9997493028640747,
 0.34968218207359314,
 0.3382493853569031,
 -0.786911129951477,
 0.9995999336242676,
 0.33503133058547974,
 0.3405217230319977,
 -0.7875194549560547,
 0.9997124671936035,
 0.502251923084259,
 0.3675001859664917,
 -0.6513850092887878,
 0.9997104406356812,
 0.3372694253921509,
 0.3799232840538025,
 -0.3661612868309021,
 0.9993034601211548,
 0.42581748962402344,
 0.4679778516292572,
 -0.7631285786628723,
 0.9999024271965027,
 0.36058521270751953,
 0.473202645778656,
 -0.6838585138320923,
 0.9998311400413513,
 0.6713072657585144,
 0.733101487159729,
 -0.5217053294181824

# 3. Training the model

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [9]:
df = pd.read_csv("coords.csv")

In [10]:
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,Happy,0.384617,0.322729,-0.863890,0.999683,0.420957,0.268093,-0.840398,0.999702,0.440225,...,-0.009025,0.0,0.460277,0.267971,-0.004765,0.0,0.466200,0.263719,-0.005456,0.0
1,Happy,0.394939,0.321733,-0.852800,0.999668,0.426580,0.267077,-0.826600,0.999664,0.446114,...,-0.006157,0.0,0.461741,0.259187,0.000391,0.0,0.467443,0.253382,0.000196,0.0
2,Happy,0.399921,0.320605,-0.799390,0.999665,0.428417,0.266148,-0.775850,0.999638,0.447719,...,-0.006671,0.0,0.460707,0.258572,-0.000163,0.0,0.466254,0.252646,-0.000295,0.0
3,Happy,0.405029,0.320234,-0.769828,0.999663,0.431008,0.265737,-0.749469,0.999610,0.450104,...,-0.005327,0.0,0.464590,0.258797,0.002500,0.0,0.470098,0.251838,0.002589,0.0
4,Happy,0.408529,0.319356,-0.746761,0.999661,0.432949,0.264793,-0.726220,0.999582,0.451852,...,-0.005108,0.0,0.469811,0.255654,0.003898,0.0,0.475156,0.248541,0.004069,0.0


In [17]:
df[df['class']=='Winning']

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
284,Winning,0.441137,0.666129,-0.791712,0.999592,0.466263,0.618982,-0.760779,0.999092,0.480207,...,-0.002153,0.0,0.482003,0.604836,-0.004818,0.0,0.487225,0.599934,-0.005975,0.0
285,Winning,0.492343,0.666716,-0.794794,0.999560,0.515076,0.623053,-0.759815,0.999092,0.529768,...,-0.003057,0.0,0.554600,0.630891,0.004186,0.0,0.558827,0.626944,0.004005,0.0
286,Winning,0.513182,0.669709,-0.795878,0.999550,0.534919,0.629341,-0.755327,0.999106,0.549350,...,-0.002900,0.0,0.559135,0.629791,0.004248,0.0,0.563381,0.626241,0.004034,0.0
287,Winning,0.521317,0.673007,-0.796527,0.999551,0.543582,0.633126,-0.754130,0.999145,0.558801,...,-0.002850,0.0,0.563667,0.629704,0.004229,0.0,0.567943,0.626198,0.004010,0.0
288,Winning,0.527825,0.672980,-0.749958,0.999557,0.547923,0.632879,-0.703969,0.999179,0.562581,...,-0.002847,0.0,0.566912,0.627673,0.004385,0.0,0.571098,0.624237,0.004161,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534,Winning,0.379153,0.671279,-0.824848,0.997462,0.407934,0.611139,-0.818989,0.996456,0.423179,...,-0.008366,0.0,0.446095,0.595919,-0.013760,0.0,0.450853,0.588689,-0.015072,0.0
535,Winning,0.325317,0.365976,-0.541835,0.999976,0.354183,0.289572,-0.479076,0.999968,0.375206,...,-0.016548,0.0,0.392981,0.289194,-0.010134,0.0,0.397360,0.282464,-0.010931,0.0
536,Winning,0.347556,0.375456,-0.571756,0.999974,0.373050,0.303261,-0.513698,0.999963,0.392564,...,-0.015392,0.0,0.412560,0.315234,-0.010160,0.0,0.416993,0.309700,-0.011048,0.0
537,Winning,0.348621,0.376574,-0.568086,0.999973,0.373164,0.308331,-0.514107,0.999957,0.392834,...,-0.018798,0.0,0.393335,0.317553,-0.021522,0.0,0.397803,0.311829,-0.023017,0.0


In [11]:
x = df.drop('class', axis= 1) # features
y = df['class'] # target value

In [12]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1234)

In [13]:
y_train

93       Happy
347    Winning
351    Winning
24       Happy
514    Winning
        ...   
279        Sad
372    Winning
204        Sad
53       Happy
294    Winning
Name: class, Length: 377, dtype: object

In [14]:
from sklearn.pipeline import make_pipeline # allows you to build a ML pipeline
from sklearn.preprocessing import StandardScaler # Normalizes your data

# Diferent algorithms for diversifing your model training (?)
from sklearn.linear_model import LogisticRegression, RidgeClassifier 
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [25]:
pipelines = {
    'lr': make_pipeline(StandardScaler(), LogisticRegression()),
    'rc': make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf': make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb': make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [26]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(x_train, y_train)
    fit_models[algo] = model

C:\Users\julia\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [27]:
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [28]:
fit_models['rc'].predict(x_test)

array(['Sad', 'Happy', 'Happy', 'Happy', 'Winning', 'Happy', 'Winning',
       'Happy', 'Happy', 'Happy', 'Happy', 'Winning', 'Sad', 'Happy',
       'Happy', 'Sad', 'Winning', 'Sad', 'Winning', 'Winning', 'Winning',
       'Sad', 'Winning', 'Happy', 'Winning', 'Happy', 'Winning', 'Sad',
       'Happy', 'Sad', 'Sad', 'Happy', 'Happy', 'Happy', 'Winning',
       'Winning', 'Sad', 'Happy', 'Happy', 'Winning', 'Winning',
       'Winning', 'Happy', 'Sad', 'Winning', 'Happy', 'Winning',
       'Winning', 'Happy', 'Winning', 'Happy', 'Winning', 'Sad',
       'Winning', 'Winning', 'Winning', 'Winning', 'Winning', 'Sad',
       'Happy', 'Winning', 'Sad', 'Happy', 'Winning', 'Happy', 'Happy',
       'Winning', 'Sad', 'Winning', 'Happy', 'Winning', 'Winning',
       'Winning', 'Winning', 'Winning', 'Happy', 'Winning', 'Winning',
       'Winning', 'Happy', 'Winning', 'Winning', 'Happy', 'Happy', 'Sad',
       'Sad', 'Winning', 'Happy', 'Happy', 'Winning', 'Happy', 'Winning',
       'Happy', 'Winni

# 4. Evaluate and serialize model

In [15]:
from sklearn.metrics import accuracy_score # accuracy metrics
import pickle

In [16]:
for algo, model in fit_models.items():
    yhat = model.predict(x_test)
    print(algo, accuracy_score(y_test, yhat))

NameError: name 'fit_models' is not defined

In [82]:
fit_models['rf'].predict(x_test)

array(['Sad', 'Happy', 'Happy', 'Happy', 'Winning', 'Happy', 'Winning',
       'Happy', 'Happy', 'Happy', 'Happy', 'Winning', 'Sad', 'Happy',
       'Happy', 'Sad', 'Winning', 'Sad', 'Winning', 'Winning', 'Winning',
       'Sad', 'Winning', 'Happy', 'Winning', 'Happy', 'Winning', 'Sad',
       'Happy', 'Sad', 'Sad', 'Happy', 'Happy', 'Happy', 'Winning',
       'Winning', 'Sad', 'Happy', 'Happy', 'Winning', 'Winning',
       'Winning', 'Happy', 'Sad', 'Winning', 'Happy', 'Winning',
       'Winning', 'Happy', 'Winning', 'Happy', 'Winning', 'Sad',
       'Winning', 'Winning', 'Winning', 'Winning', 'Winning', 'Sad',
       'Happy', 'Winning', 'Sad', 'Happy', 'Winning', 'Happy', 'Happy',
       'Winning', 'Sad', 'Winning', 'Happy', 'Winning', 'Winning',
       'Winning', 'Winning', 'Winning', 'Happy', 'Winning', 'Winning',
       'Winning', 'Happy', 'Winning', 'Winning', 'Happy', 'Happy', 'Sad',
       'Sad', 'Winning', 'Happy', 'Happy', 'Winning', 'Happy', 'Winning',
       'Happy', 'Winni

In [83]:
y_test

181        Sad
138      Happy
11       Happy
21       Happy
523    Winning
        ...   
484    Winning
122      Happy
477    Winning
469    Winning
134      Happy
Name: class, Length: 162, dtype: object

In [38]:
with open('body_language.pkl','wb') as f:
    pickle.dump(fit_models['rf'], f)

# Make detections

In [1]:
 with open('body_language.pkl', 'rb') as f:
        model = pickle.load(f)

NameError: name 'pickle' is not defined

In [34]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())])

In [35]:
cap = cv2.VideoCapture(0)

#Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # Make Detections
        results = holistic.process(image)
        
        # Recolor back
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS, 
                                  mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                  mp_drawing.DrawingSpec(color=(80,256,121), thickness=1,circle_radius=1)
                                 )
        # Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                  mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(80,256,121), thickness=2,circle_radius=2)
                                 )
        # Left hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                  mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(80,256,121), thickness=2,circle_radius=2)
                                 )
        # Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                  mp_drawing.DrawingSpec(color=(245,117,10), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(245,256,121), thickness=2,circle_radius=2)
                                 )
        # Export coordinates
        try:
            # Extract pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Extract face landmarks 
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # Concatenate rows
            row = pose_row + face_row
            
#             # Append class name
#             row.insert(0, class_name)
            
            # Make detections
            x = pd.DataFrame([row])
            body_language_class = model.predict(x)[0]
            body_language_prob = model.predict_proba(x)[0]
            print(body_language_class, body_language_prob)
            
            # Grab ear coordinates
            display_coords = tuple(np.multiply(
                                np.array(
                                    (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x,
                                     results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                                     , [640, 480]).astype(int))
            
            cv2.rectangle(image,
                         (display_coords[0], display_coords[1]+5),
                         (display_coords[0]+ len(body_language_class)*20, display_coords[1]-30),
                         (245, 117, 16), -1)
            cv2.putText(image, body_language_class, display_coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
#             # Export to .CSV
#             with open('coords.csv', mode='a', newline='') as f:
#                 csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#                 csv_writer.writerow(row)
            
        except:
            pass
        
        cv2.imshow('Raw webcam feed', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    
cap.release()
cv2.destroyAllWindows()

Winning [0.11 0.15 0.74]
Winning [0.1  0.13 0.77]
Winning [0.12 0.11 0.77]
Winning [0.11 0.11 0.78]
Winning [0.1  0.02 0.88]
Winning [0.11 0.02 0.87]
Winning [0.06 0.06 0.88]
Winning [0.01 0.1  0.89]
Winning [0.01 0.08 0.91]
Winning [0.01 0.09 0.9 ]
Winning [0.01 0.09 0.9 ]
Winning [0.01 0.07 0.92]
Winning [0.01 0.07 0.92]
Winning [0.01 0.07 0.92]
Winning [0.01 0.03 0.96]
Winning [0.01 0.03 0.96]
Winning [0.01 0.02 0.97]
Winning [0.01 0.02 0.97]
Winning [0.   0.03 0.97]
Winning [0.   0.03 0.97]
Winning [0.   0.02 0.98]
Winning [0.   0.02 0.98]
Winning [0.   0.02 0.98]
Winning [0.   0.02 0.98]
Winning [0.   0.01 0.99]
Winning [0.   0.01 0.99]
Winning [0.   0.03 0.97]
Winning [0.   0.02 0.98]
Winning [0.   0.03 0.97]
Winning [0.   0.02 0.98]
Winning [0.   0.01 0.99]
Winning [0. 0. 1.]
Winning [0.   0.01 0.99]
Winning [0.02 0.01 0.97]
Winning [0.01 0.02 0.97]
Winning [0.02 0.02 0.96]
Winning [0.02 0.05 0.93]
Winning [0.01 0.04 0.95]
Winning [0.01 0.03 0.96]
Winning [0.01 0.13 0.86]
Winnin